# DECSKS-30 -- Constructing FD weight matrices with only centered stencils

## organization of notebook

    (1) we step through the assembly of a finite difference weight matrix that uses PBCs so that
        off-grid points are sampled on "the other side" of the domain. This is exclusively for periodic
        problem so that we can compare the fidelity of finite difference correctiosn with Fourier
        corrections which both can be compared when applied to periodic problems.
        
    (2) we step through the straightforward assembly of a finite difference weight matrix that samples
        ghost points whenever the stencil goes off-grid. This is actually trivial, we just present the
        routine here for completeness and organization
        
There are two routines that would need to be extended, which have prototypes, the first is used for the first derivative which we need to apply at each timestep to get the acceleration Ex = -W.dot(phi), the second is
a routine that loops through all orders q = 0, 1, ... , N-1. In this regard, the second routine is basically a loop of the first routine, so the detials of their working is discernible once we figure out one. The two routines are as follows:

    lib.read.assemble_finite_difference_weight_matrix_const_dn_const_LTE(
                                             BC,
                                             z_str,
                                             zN,
                                             FD_scheme_const_dn,
                                             dn = 1,
                                             LTE = 6
                                             )
                                             
    lib.read.assemble_finite_difference_weight_matrix(
                                             BC,
                                             z_str,
                                             zN,
                                             N,
                                             FD_schemes
                                             ):


<b>we work on the routine <code>lib.read.assemble_finite_difference_weight_matrix_const_dn_const_LTE</code> as it is natural to work through the details of just one derivative, then extend the results for the second routine listed above for all derivative order dn.</b>

the routine that assembles the matrix is the following:

### <code>lib.read.assemble_finite_difference_weight_matrix_const_dn_const_LTE</code>

In [ ]:
def assemble_finite_difference_weight_matrix_const_dn_const_LTE(BC,
                                                                z_str,
                                                                zN,
                                             FD_scheme_const_dn,
                                             dn = 1,
                                             LTE = 6
                                             ):
    """Assembles a matrix corresponding to the weights of in
    the finite difference computation of derivatives, i.e.
    assembles the weight matrix W, giving the difference matrix d
    for the q-th derivative:

        1 / x.width ** q W[q,:,:].dot(f) =  d[q,:,:]

                                    i.e. W are the difference
                                    coefficients, which do not
                                    contain the width of the
                                    abscissa value, e.g. x.width

    where f and df are vectors of length z.N in the 1D case.

    inputs:
    BC -- (dict) boundary condition dictionary
    z_str -- (str) z.str attribute, e.g. x.str = 'x', vx.str = 'vx', ...
    zN -- (int) number of active grid points for the phase space variable z
    N -- (int) global error on the advection algorithm, specified in etc/params.dat
    FD_schemes -- (dict) dictionary containing all schemes for all dn, handedness,
        and asymmetry
    outputs:
    Wz -- (ndarray, ndim=3) Wz[dn, zN, zN] where each 2D matrix Wz[dn,:,:]
          is the weight matrix W corresponding to the dn-th derivative in
          the context of the above equation.
    """
    imax = zN - 1
    W = np.zeros([zN, zN])

    # local copy of all schemes pertaining to derivative order dn
    FD_scheme = FD_scheme_const_dn['dn' + str(dn)]['LTE' + str(LTE)]
    stencil_size = LTE + dn
    stencil_center = stencil_size // 2

    for i in range(zN):

        if i < stencil_center:
            handedness = 'forward'
            asymmetry = str(i)
        elif imax - i < stencil_center:
            handedness = 'backward'
            asymmetry = str(imax - i)
        else:
            if np.mod(stencil_size,2) == 1: # only schemes with an odd number
                                            # stencils have a central scheme
                handedness = 'central'
                asymmetry = str(0)
            else:
                handedness = 'forward'  # or equivalently could call it 'backward'
                                        # and take assymetry = stencil_center+1, same thing

                asymmetry = str(stencil_center - 1)

        w = FD_scheme[handedness][asymmetry]['w']
        stencil = FD_scheme[handedness][asymmetry]['stencil']

        W[i, i + np.array(stencil)] = w # load all weights at once into W for row i

    return W


One example will be sufficient to illustrate all the requirements

### Example: $N_x = 5, i = 0,1, 2, 3, 4$, centered stencil $S = \{-2,-1,0,1,2\}$

Thus the corresponding weights are indexed as $\underline{w} = [w_0, w_1, w_2, w_3, w_4]$

We introduce quantities $N_{off}, N_{on}$ which indicate the number of grid points according to the stencil and current row/column $i$ (each stencil is centered about the matrix location $[i,i]$) that are seen to be referencing data that is off-grid and on-grid respectively (e.g. $i < 0$ or $i > N_x-1$).

We require the following assignments of the weight list $\underline{w}$ to a difference matrix $\underline{\underline{W}} = \underline{\underline{W}}_{5\times 5}$, where we step through $i = 0,1 ,2,3 ,4$,k we use python style indexing below for clarity:

$\boxed{i = 0}$:

$$S + i = \text{gridpoints referenced } = i\in [-2, -1, 0, 1, 2]$$

$$\Rightarrow N_{off} = 2, \quad \text{gridpoints } i = -2, -1$$
$$\Rightarrow N_{on} = 3, \quad \text{gridpoints } i = 0,1,2$$

notice that $N_{on} + N_{off} = |S| = 5$, the stencil size where we use $|\cdot | $ to indicate cardinality of the set.

\begin{eqnarray*}
W[0,-2]  & = & w[0] \\
W[0,-1]  & = & w[1] \\
W[0,0:3]  & = & w[2:]
\end{eqnarray*}


$\boxed{i = 1}$:

$$S + i = \text{gridpoints referenced } = i\in [-1, 0, 1, 2,3]$$

$$\Rightarrow N_{off} = 1, \quad \text{gridpoints } i = -2$$
$$\Rightarrow N_{on} = 4, \quad \text{gridpoints } i = 0,1,2,3$$

notice that $N_{on} + N_{off} = |S| = 5$, we have the assignments:

\begin{eqnarray*}
W[1,-1]  & = & w[0] \\
W[1,0:4]  & = & w[1:]
\end{eqnarray*}


$\boxed{i = 2}$:

$$S + i = \text{gridpoints referenced } = i\in [0, 1, 2,3,4]$$

$$\Rightarrow N_{off} = 0$$
$$\Rightarrow N_{on} = 5, \quad \text{gridpoints } i = 0,1,2,3,4$$

notice that $N_{on} + N_{off} = |S| = 5$, we have the assignments:

\begin{eqnarray*}
W[1,0:5]  & = & w[:]
\end{eqnarray*}

$\boxed{i = 3}$:

$$S + i = \text{gridpoints referenced } = i\in [1,2,3,4,5]$$

$$\Rightarrow N_{off} = 1, \quad \text{gridpoints } i = 5$$
$$\Rightarrow N_{on} = 4, \quad \text{gridpoints } i = 1,2,3,4$$

notice that $N_{on} + N_{off} = |S| = 5$, we have the assignments:

\begin{eqnarray*}
W[3,0]  & = & w[4] \\
W[3,1:5]  & = & w[:5]
\end{eqnarray*}

where W[3,1:5] might like to be written as W[3,Nx-N_on:Nx], where Nx = 5, N_on = 4

$\boxed{i = 4}$:

$$S + i = \text{gridpoints referenced } = i\in [2,3,4,5,6]$$

$$\Rightarrow N_{off} = 2, \quad \text{gridpoints } i = 5,6$$
$$\Rightarrow N_{on} = 4, \quad \text{gridpoints } i = 2,3,4$$

notice that $N_{on} + N_{off} = |S| = 5$, we have the assignments:

\begin{eqnarray*}
W[4,0]  & = & w[3] \\
W[4,1]  & = & w[4] \\
W[4,2:5]  & = & w[:3]
\end{eqnarray*}



This generalizes readily if we use Python to construct it, negative indices permit easy array assignments for any central node (where the stencil informs to sample all on-grid points) or left edge node (where the stencil niforms to sample off-grid points which are represented as negative indices):

if $i$ samples left of the grid of is a central node, then assign for each row $i$: 

    W[i, i + stencil] = w # stencil and w are 1D arrays
    
if $i$ samples to off-grid on the right side, then assign for each such row $i$:

    N_off = number of off-grid references per the stencil
    
    W[i, i + stencil[:N_off] ]     = w[:N_off]
    W[i, i + stencil[N_off:] - Nx] = w[N_off:] # the slice i + stencil[N_off:] - Nx takes Nx -> 0, Nx+1 -> 1, etc.
    
We show a quick proofing below

#### example of odd number stencil (centered scheme exists)

In [25]:
import numpy as np

Nx = 12
S = np.arange(-2,3,1)
stencil_size  = len(S)
stencil_center = stencil_size // 2
w = [-2,-1,999,1,2]
W = np.zeros((Nx,Nx))

print "to make it obvious the correct weights have been assigned, we choose weights"
print "that are recognizable (though unphysical):\n"
print "w = " 
print w
  
print "\nWeights are assigned then as"
for i in range(Nx):
    if Nx - i <= stencil_center:
        N_off = (Nx - 1) - i
        W[i, i + S[:N_off]]  = w[:N_off]
        W[i, i + S[N_off:] - Nx]  = w[N_off:]
        
    else:
        W[i, i + S]  = w
print W

to make it obvious the correct weights have been assigned, we choose weights
that are recognizable (though unphysical):

w = 
[-2, -1, 999, 1, 2]

Weights are assigned then as
[[ 999.    1.    2.    0.    0.    0.    0.    0.    0.    0.   -2.   -1.]
 [  -1.  999.    1.    2.    0.    0.    0.    0.    0.    0.    0.   -2.]
 [  -2.   -1.  999.    1.    2.    0.    0.    0.    0.    0.    0.    0.]
 [   0.   -2.   -1.  999.    1.    2.    0.    0.    0.    0.    0.    0.]
 [   0.    0.   -2.   -1.  999.    1.    2.    0.    0.    0.    0.    0.]
 [   0.    0.    0.   -2.   -1.  999.    1.    2.    0.    0.    0.    0.]
 [   0.    0.    0.    0.   -2.   -1.  999.    1.    2.    0.    0.    0.]
 [   0.    0.    0.    0.    0.   -2.   -1.  999.    1.    2.    0.    0.]
 [   0.    0.    0.    0.    0.    0.   -2.   -1.  999.    1.    2.    0.]
 [   0.    0.    0.    0.    0.    0.    0.   -2.   -1.  999.    1.    2.]
 [   2.    0.    0.    0.    0.    0.    0.    0.   -2.   -1.  999.    1.]

So we have the summary construction

<table "width = 95%">
<tr><td><b>Setting up a periodic finite difference weight matrix with 0-based indexing</b></td></tr>
<tr><td>
$${}$$
For $N_x$ gridpoints, we define:
<ul>
<li>an <b>odd</b> stencil $\underline{S} = (S_s)_{s=0}^{N_S-1} = (S_0, S_1, S_2, \ldots , S_{N_S-1}) = \left(-\lfloor\frac{N_S}{2}\rfloor, -\lfloor\frac{N_S}{2}\rfloor + 1, \ldots , -1, 0, 1, \ldots , \lfloor\frac{N_S}{2}\rfloor - 1, \lfloor\frac{N_S}{2}\rfloor\right)$
<ul>
<li>of total number $N_S = 2\cdot \lfloor N_S / 2\rfloor + 1$ if odd
<li>with an (even) index center at $\lfloor \frac{N_S}{2}\rfloor$, i.e. $S_{\lfloor N_S / 2\rfloor} = 0$
</ul>
<li>corresponding weights $\underline{w} = (w_s)_{s=0}^{N_S-1} = (w_0, w_1, w_2, \ldots , w_{N_S-1}$ that go with each $S_s$
</ul>
$${}$$
    A finite difference estimate for the $q$th derivative of a function $f$ at the gridvalue $x_i$ where $i$ is referred to as its gridpoint, is computed as a linear combination of weights $(\Delta x)^q \partial_x^q = \sum_s w_s f(x_{(i+S_s)\text{mod } N_x})$ where per the modular arithmetic is used to communicate periodicity for any off-gridpoint sampling that a stencil requires for $i$ close enough to the edges
    
We create a 2D finite difference weight matrix $\underline{\underline{W}} = \underline{\underline{W}}_{N_x\times N_x}$ and store the weights in each row $i$ along columns that span the gridpoints per the periodic boundary condition (an off-grid reference is simply taken to wrap around the grid and sample from the other side) according toL
$${}$$
For each row (gridpoint) $i$:
$${}$$
$\quad$if $N_x-i \leq \lfloor\frac{|S|}{2}\rfloor$ then the stencil will sample off-grid values on the right side
$${}$$
$\qquad N_{off} = (N_x-1) - i$ is the total number of gridpoints sampled off-grid on the right
$${}$$
$\qquad$vector assignment: $\underline{\underline{W}}[\,i, i + \underline{S}[:N_{off}]\,]  = \underline{w}[:N_{off}]$
$${}$$
$\qquad$vector assignment: $\underline{\underline{W}}[\,i, i + \underline{S}[N_{off}:] - N_x\,]  = \underline{w}[N_{off}:]$
$${}$$
$${}$$
$\quad$else 
$${}$$
$\qquad$ (then the stencil will sample off-grid values on the left-side where negative indexing 
$${}$$
$\quad \qquad$has a meaning, or is a central gridpoint where all references are on-grid)
$${}$$
$\qquad$vector assignment: $\underline{\underline{W}}[\,i, i + \underline{S}\,]  = \underline{w}$
$${}$$
</td></tr></table>

###<font color ="purple">This also works for even numbered stencils (by virtue of negative indexing, the roles of each assignment get swapped but they complete as expected (we show this below)</font>

#### example of even number stencil (centered scheme does not exist, the decision in DECSKS is always to choose a $f-1$ scheme (same as $b+1$ given two boundaries)

In [28]:
import numpy as np

Nx = 12
S = np.arange(-1,3,1)
stencil_size  = len(S)
stencil_center = stencil_size // 2
w = [-1,999,1,2]
W = np.zeros((Nx,Nx))

print "to make it obvious the correct weights have been assigned, we choose weights"
print "that are recognizable (though unphysical):\n"
print "w = " 
print w

print "\nWeights are assigned then as"
for i in range(Nx):
    if Nx - i <= stencil_center:
        N_off = (Nx - 1) - i
        W[i, i + S[:N_off]]  = w[:N_off]
        W[i, i + S[N_off:] - Nx]  = w[N_off:]
        
    else:
        W[i, i + S]  = w
print W

to make it obvious the correct weights have been assigned, we choose weights
that are recognizable (though unphysical):

w = 
[-1, 999, 1, 2]

Weights are assigned then as
[[ 999.    1.    2.    0.    0.    0.    0.    0.    0.    0.    0.   -1.]
 [  -1.  999.    1.    2.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.   -1.  999.    1.    2.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.   -1.  999.    1.    2.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.   -1.  999.    1.    2.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.   -1.  999.    1.    2.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.   -1.  999.    1.    2.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.   -1.  999.    1.    2.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.   -1.  999.    1.    2.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.   -1.  999.    1.    2.]
 [   2.    0.    0.    0.    0.    0.    0.    0.    0.   -1.  999.    1.]
 [ 

# Finalized new routines (in <code>lib.read</code> module)

In [ ]:
def assemble_finite_difference_weight_matrix_const_dn_const_LTE(BC,
                                                                z_str,
                                                                zN,
                                             FD_scheme_const_dn,
                                             dn = 1,
                                             LTE = 6
                                             ):
    """Assembles a matrix corresponding to the weights of in
    the finite difference computation of derivatives, i.e.
    assembles the weight matrix W, giving the difference matrix d
    for the q-th derivative:

        1 / x.width ** q W[q,:,:].dot(f) =  d[q,:,:]

                                    i.e. W are the difference
                                    coefficients, which do not
                                    contain the width of the
                                    abscissa value, e.g. x.width

    where f and df are vectors of length z.N in the 1D case.

    inputs:
    BC -- (dict) boundary condition dictionary
    z_str -- (str) z.str attribute, e.g. x.str = 'x', vx.str = 'vx', ...
    zN -- (int) number of active grid points for the phase space variable z
    N -- (int) global error on the advection algorithm, specified in etc/params.dat
    FD_schemes -- (dict) dictionary containing all schemes for all dn, handedness,
        and asymmetry
    outputs:
    Wz -- (ndarray, ndim=3) Wz[dn, zN, zN] where each 2D matrix Wz[dn,:,:]
          is the weight matrix W corresponding to the dn-th derivative in
          the context of the above equation.
    """
    imax = zN - 1

    if BC['f'][z_str]['type'] == 'periodic':
        # set up matrix that uses PBC in sampling; off-grid referencing
        # samples from the opposite side of the domain per periodic BC
        W = np.zeros([zN, zN])

        # local copy of all schemes pertaining to derivative order dn
        FD_scheme = FD_scheme_const_dn['dn' + str(dn)]['LTE' + str(LTE)]
        stencil_size = LTE + dn
        stencil_center = stencil_size // 2

        # CHOOSE SCHEME, if odd choose central, elif even choose most centered
        if np.mod(stencil_size,2) == 1: # only schemes with an odd number
                                        # stencils have a central scheme
            handedness = 'central'
            asymmetry = str(0)
        else:
            handedness = 'forward'      # or equivalently could call it 'backward'
                                        # and take assymetry = stencil_center+1, same thing

            asymmetry = str(stencil_center - 1)

        w = FD_scheme[handedness][asymmetry]['w']
        stencil = FD_scheme[handedness][asymmetry]['stencil']

        # CONSTRUCT WEIGHT MATRIX -- off-grid points will sample "other side" per PBC
        for i in range(zN): # loop through rows whose row label corresponds to gridpoint of interest

            if zN - i <= stencil_center: # then gridpoint is close enough to right-edge that
                                         # it samples off-grid on the right side


                N_offgrid = zN - 1 - i         # number of off-grid points the
                                               # stencil references when stencil is odd, otherwise
                                               # the meaning is N_ongrid for even stencil but
                                               # the following still works given negative indexing

                W[i, i + np.array(stencil)[:N_offgrid]     ] = w[:N_offgrid]
                W[i, i + np.array(stencil)[N_offgrid:] - zN] = w[N_offgrid:]

            else:
                W[i, i + np.array(stencil) ] = w

    else: # explicit finite differencing whose stencil shifts near edges to evade
          # off-grid indexing (using more sided schemes)

        imax = zN - 1
        W = np.zeros([zN, zN])

        # local copy of all schemes pertaining to derivative order dn
        FD_scheme = FD_scheme_const_dn['dn' + str(dn)]['LTE' + str(LTE)]
        stencil_size = LTE + dn
        stencil_center = stencil_size // 2

        for i in range(zN):

            if i < stencil_center:
                handedness = 'forward'
                asymmetry = str(i)
            elif imax - i < stencil_center:
                handedness = 'backward'
                asymmetry = str(imax - i)
            else:
                if np.mod(stencil_size,2) == 1: # only schemes with an odd number
                                                # stencils have a central scheme
                    handedness = 'central'
                    asymmetry = str(0)
                else:
                    handedness = 'forward'  # or equivalently could call it 'backward'
                                            # and take assymetry = stencil_center+1, same thing

                    asymmetry = str(stencil_center - 1)

            w = FD_scheme[handedness][asymmetry]['w']
            stencil = FD_scheme[handedness][asymmetry]['stencil']

            W[i, i + np.array(stencil)] = w # load all weights at once into W_dn

    return W

and

In [ ]:
def assemble_finite_difference_weight_matrix(BC,
                                             z_str,
                                             zN,
                                             N,
                                             FD_schemes
                                             ):
    """Assembles a matrix corresponding to the weights of in
    the finite difference computation of derivatives, i.e.
    assembles the weight matrix W, giving the difference matrix d
    for the q-th derivative:

        1 / x.width ** q W[q,:,:].dot(f) =  d[q,:,:]

                                    i.e. W are the difference
                                    coefficients, which do not
                                    contain the width of the
                                    abscissa value, e.g. x.width

    where f and df are vectors of length z.N in the 1D case.

    inputs:
    zN -- (int) number of active grid points for the phase sapce variable z
    N -- (int) global error on the advection algorithm, specified in etc/params.dat
    FD_schemes -- (dict) dictionary containing all schemes for all dn, handedness,
        and asymmetry
    outputs:
    Wz -- (ndarray, ndim=3) Wz[dn, zN, zN] where each 2D matrix Wz[dn,:,:]
          is the weight matrix W corresponding to the dn-th derivative in
          the context of the above equation.
    """
    if BC['f'][z_str]['type'] == 'periodic':

        imax = zN - 1
        Wz = np.zeros([N, zN, zN]) # includes zeroeth order derivative container (not used)
        # i.e. Wz[q,:,:] is for the q-th derivative with this dummy zero index created

        for dn in range(1,N):
            W_dn = np.zeros([zN, zN])
            p = N - dn     # LTE of scheme on dn-th derivative decreases with dn
                           # given what is needed is LTE[z.width ** dn * dnf] = O(N)
                           # rather than LTE on just the derivative

            # local copy of all schemes pertaining to derivative order dn
            FD_schemes_dn = FD_schemes['dn' + str(dn)]
            stencil_size = p + dn
            stencil_center = stencil_size // 2

            # choose as centered a scheme as possible given the stencil size
            if np.mod(stencil_size,2) == 1: # stencil size is odd
                handedness = 'central'
                asymmetry = str(0)
            else:                           # stencil size is even
                handedness = 'forward'
                asymmetry = str(stencil_center - 1)

            FD_scheme = FD_schemes_dn[handedness][asymmetry]
            w = FD_scheme['w']
            stencil = FD_scheme['stencil']


            # CONSTRUCT WEIGHT MATRIX -- off-grid points will sample "other side" per PBC
            for i in range(zN):
                if zN - i <= stencil_center: # then gridpoint is close enough to right-edge that
                                             # it samples off-grid on the right side

                    N_offgrid = zN - 1 - i         # number of off-grid points the
                                                   # stencil references when stencil is odd, otherwise
                                                   # the meaning is N_ongrid for even stencil but
                                                   # the following still works given negative indexing

                    W_dn[i, i + np.array(stencil)[:N_offgrid]     ] = w[:N_offgrid]
                    W_dn[i, i + np.array(stencil)[N_offgrid:] - zN] = w[N_offgrid:]

                else:
                    W_dn[i, i + np.array(stencil) ] = w

            Wz[dn,:,:] = W_dn

    else:

        imax = zN - 1
        Wz = np.zeros([N, zN, zN]) # includes zeroeth order derivative container (not used)
        # i.e. Wz[q,:,:] is for the q-th derivative with this dummy zero index created

        for dn in range(1,N):
            W_dn = np.zeros([zN, zN])
            p = N - dn     # LTE of scheme on dn-th derivative decreases with dn
                           # given what is needed is LTE[z.width ** dn * dnf] = O(N)
                           # rather than LTE on just the derivative

            # local copy of all schemes pertaining to derivative order dn
            FD_schemes_dn = FD_schemes['dn' + str(dn)]
            stencil_size = p + dn
            stencil_center = stencil_size // 2

            for i in range(zN):

                if i < stencil_center:
                    handedness = 'forward'
                    asymmetry = str(i)
                elif imax - i < stencil_center:
                    handedness = 'backward'
                    asymmetry = str(imax - i)
                else:
                    if np.mod(stencil_size,2) == 1:
                        handedness = 'central'
                        asymmetry = str(0)
                    else:
                        handedness = 'forward'
                        asymmetry = str(stencil_center - 1)

                FD_scheme = FD_schemes_dn[handedness][asymmetry]
                w = FD_scheme['w']
                stencil = FD_scheme['stencil']

                W_dn[i, i + np.array(stencil)] = w # load all weights at once into W_dn

            Wz[dn,:,:] = W_dn

    return Wz